In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install datasets
!pip install transformers

In [ ]:
# -*- coding: utf-8 -*-
"""NLU3_pretrain_bert_14.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1I4acc-cQiBEqacO81ksrYwLWnNWKVR-5
"""



import transformers

print(transformers.__version__)

model_checkpoint =  "bert-base-multilingual-cased"
batch_size = 16

# Give the path for train data
from pathlib import Path
import json
path = Path('/content/drive/MyDrive/NLU3/train_samples.json')
#path = Path('squad/train-v2.0.json')
# Open .json file
with open(path, 'rb') as f:
    squad_dict = json.load(f)


texts = []
queries = []
answers = []
ID = []
title = []

# Search for each passage, its question and its answer
for group in squad_dict['data']:
    ti = group["title"]
    for passage in group['paragraphs']:
       context = passage['context']
        for qa in passage['qas']:
            question = qa['question']
            id = qa["id"]
            answer =  qa['answers']
            if len(answer ) == 0 :
              answer1 = {'text':[""] , 'answer_start':[]}
              answers.append(answer1)
              title.append(ti)
              texts.append(context)
              queries.append(question)
              ID.append(id)
            else:
              for i in range(len(answer)):
                  answer1 = {'text':[answer[i]['text']] , 'answer_start':[answer[i]['answer_start']]}
                  answers.append(answer1)
                  title.append(ti)
                  texts.append(context)
                  queries.append(question)
                  ID.append(id)

train_context, train_question, train_answers, train_id, train_title = texts, queries, answers, ID, title

# Give the path for validation data
path = Path('/content/drive/MyDrive/NLU3/validation_samples.json')
#path = Path('squad/dev-v2.0.json')
# Open .json file
with open(path, 'rb') as f:
    squad_dict = json.load(f)

texts = []
queries = []
answers = []
ID = []
title = []

# Search for each passage, its question and its answer
for group in squad_dict['data']:
    ti = group["title"]
    for passage in group['paragraphs']:
        context = passage['context']
        for qa in passage['qas']:
            question = qa['question']
            id = qa["id"]
            answer =  qa['answers']
            if len(answer ) == 0 :
              answer1 = {'text':[""] , 'answer_start':[]}
              answers.append(answer1)
              title.append(ti)
              texts.append(context)
              queries.append(question)
              ID.append(id)
            else:
              for i in range(len(answer)):
                  answer1 = {'text':[answer[i]['text']] , 'answer_start':[answer[i]['answer_start']]}
                  answers.append(answer1)
                  title.append(ti)
                  texts.append(context)
                  queries.append(question)
                  ID.append(id)
from datasets import load_dataset
val_context, val_question, val_answers, val_id, val_title = texts, queries, answers, ID, title
datasets = load_dataset("squad_v2" )
train_s = datasets["train"][:20000]
val_s = datasets["validation"][:1000]
train_context, train_question, train_answers, train_id, train_title = train_context+ train_s["context"], train_question+ train_s["question"], train_answers + train_s["answers"], val_id+train_s["id"], train_title+train_s["title"]
val_context, val_question, val_answers, val_id, val_title = val_context+ val_s["context"], val_question+ val_s["question"], val_answers + val_s["answers"], val_id+val_s["id"], val_title+val_s["title"]
import pandas as pd
from datasets import Dataset
#['id', 'title', 'context', 'question', 'answers']
train_data = pd.DataFrame({'answers' : train_answers, 'context' : train_context, 'id' : train_id, 'question' : train_question, 'title' : train_title})
val_data = pd.DataFrame({'answers' : val_answers, 'context' : val_context, 'id' : val_id, 'question' : val_question, 'title' : val_title})

train_datasets = Dataset.from_pandas(train_data)
val_datasets = Dataset.from_pandas(val_data)
data = {"train" : train_datasets, "test" : val_datasets}
#vall_da = train_datasets('bookcorpus', split='train[:10')

train_datasets

from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

show_random_elements(train_datasets)

from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

max_length = 201
doc_stride = 128

for i, example in enumerate(train_datasets):
  if len(tokenizer(example["question"], example["context"])["input_ids"]) > 201:
    break
example = train_datasets[i]

len(tokenizer(example["question"], example["context"])["input_ids"])

len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

[len(x) for x in tokenized_example["input_ids"]]

for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

answers = example["answers"]
answers

len(answers["text"])

answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

pad_on_right = tokenizer.padding_side == "right"

def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

features = prepare_train_features(train_datasets[:5])
print(features)

tokenized_datasets = train_datasets.map(prepare_train_features, batched=True, remove_columns=train_datasets.column_names)
tokenized_datasets1 = val_datasets.map(prepare_train_features, batched=True, remove_columns=train_datasets.column_names)

from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

from transformers import default_data_collator

data_collator = default_data_collator

tokenized_datasets[0:15000]

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets1,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

4.15.0


Reusing dataset squad_v2 (/root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

,answers,context,id,question,title
0,"{'answer_start': [], 'text': []}","The density of paper ranges from 250 kg/m3 (16 lb/cu ft) for tissue paper to 1,500 kg/m3 (94 lb/cu ft) for some speciality paper. Printing paper is about 800 kg/m3 (50 lb/cu ft).",5ad690ae191832001aa7b220,What is the uncommon density of printing paper?,Paper
1,"{'answer_start': [183], 'text': ['Nocturne in E major Op. 62 No. 2']}","The British Library notes that ""Chopin's works have been recorded by all the great pianists of the recording era."" The earliest recording was an 1895 performance by Paul Pabst of the Nocturne in E major Op. 62 No. 2. The British Library site makes available a number of historic recordings, including some by Alfred Cortot, Ignaz Friedman, Vladimir Horowitz, Benno Moiseiwitsch, Paderewski, Arthur Rubinstein, Xaver Scharwenka and many others. A select discography of recordings of Chopin works by pianists representing the various pedagogic traditions stemming from Chopin is given by Methuen-Campbell in his work tracing the lineage and character of those traditions.",56cf5284aab44d1400b88fcb,What is the title of the earliest known recording of Chopin's work?,Frédéric_Chopin
2,"{'answer_start': [372], 'text': ['۲۱']}",در سال ۱۸۷۰ در بریتانیا، ۷۵ باشگاه وجود داشتند که به بازی راگبی مدرسه‌ای می‌پرداختند. این مدارس عبارت بودند از باشگاه راگبی بلک هیس (سال تأسیس ۱۸۵۸ که طبق سوابق مورد مناقشه به‌عنوان قدیمی‌ترین باشگاه غیر دانشگاهی راگبی که هنوز وجود دارد، شناخته می‌شود). باشگاه‌های راگبی همچنین در کشورهای ایرلند، استرالیا، کانادا و زلاند نو فعال بودند. بهرحال، تا سال ۱۸۷۱ یعنی زمانی که ۲۱ باشگاه لندن گرد آمده تا اتحادیه راگبی را تشکیل دهند، قواعد قابل قبولی برای راگبی وجود نداشت. (به‌صورت کنایه‌ای، بلاک هیس اقدام به ممنوع ساختن ضربه زدن نمود) اولین قواعد این اتحادیه در ماه ژوئن سال ۱۸۷۱ بکار گرفته شد. بر طبق این قوانین پاس دادن توپ مجاز اعلام شده بود. آن‌ها همچنین ترای را در جائیکه به سمت دروازه و در بیرون خط با توپ تماس ایجاد شود شامل نمودند، اگر چه گل‌های ناشی از امتیاز و بازی عمومی و برگشت پنالتی هنوز در مناقشه بودند.,1976075,سال ۱۸۷۱ چند باشگاه لندن گرد آمده تا اتحادیه راگبی را تشکیل دهند؟,راگبی فوتبال
3,"{'answer_start': [415], 'text': ['whelping']}","In breeding circles, a male canine is referred to as a dog, while a female is called a bitch (Middle English bicche, from Old English bicce, ultimately from Old Norse bikkja). A group of offspring is a litter. The father of a litter is called the sire, and the mother is called the dam. Offspring are, in general, called pups or puppies, from French poupée, until they are about a year old. The process of birth is whelping, from the Old English word hwelp.",56d4bdcd2ccc5a1400d831ad,What is giving birth to dogs called?,Dog
4,"{'answer_start': [516], 'text': ['Ajnanas']}","At the same time, these movements were influenced by, and in some respects continued, philosophical thought within the Vedic tradition as reflected e.g. in the Upanishads. These movements included, besides Buddhism, various skeptics (such as Sanjaya Belatthiputta), atomists (such as Pakudha Kaccayana), materialists (such as Ajita Kesakambali), antinomians (such as Purana Kassapa); the most important ones in the 5th century BCE were the Ajivikas, who emphasized the rule of fate, the Lokayata (materialists), the Ajnanas (agnostics) and the Jains, who stressed that the soul must be freed from matter. Many of these new movements shared the same conceptual vocabulary—atman (""Self""), buddha (""awakened one""), dhamma (""rule"" or ""law""), karma (""action""), nirvana (""extinguishing""), samsara (""eternal recurrence"") and yoga (""spiritual practice"").[note 24] The shramanas rejected the Veda, and the authority of the brahmans, who claimed they possessed revealed truths not knowable by any ordinary human means. Moreover, they declared that the entire Brahmanical system was fraudulent: a conspiracy of the brahmans to enrich themselves by charging exorbitant fees to perform bogus rites and give useless advice.",56d24e

loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "abs

...

  0%|          | 0/15 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "abs

Epoch,Training Loss,Validation Loss
1,0.947400,0.938813
2,0.623800,0.960728
3,0.390500,1.112153


Saving model checkpoint to bert-base-multilingual-cased-finetuned-squad/checkpoint-500
Configuration saved in bert-base-multilingual-cased-finetuned-squad/checkpoint-500/config.json
Model weights saved in bert-base-multilingual-cased-finetuned-squad/checkpoint-500/pytorch_model.bin
tokenizer config file saved in bert-base-multilingual-cased-finetuned-squad/checkpoint-500/tokenizer_config.json
Special tokens file saved in bert-base-multilingual-cased-finetuned-squad/checkpoint-500/special_tokens_map.json
Saving model checkpoint to bert-base-multilingual-cased-finetuned-squad/checkpoint-1000
Configuration saved in bert-base-multilingual-cased-finetuned-squad/checkpoint-1000/config.json
Model weights saved in bert-base-multilingual-cased-finetuned-squad/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in bert-base-multilingual-cased-finetuned-squad/checkpoint-1000/tokenizer_config.json
Special tokens file saved in bert-base-multilingual-cased-finetuned-squad/checkpoint-1000/s

TrainOutput(global_step=6108, training_loss=0.746590533753041, metrics={'train_runtime': 9455.1317, 'train_samples_per_second': 10.333, 'train_steps_per_second': 0.646, 'total_flos': 1.0022416801458912e+16, 'train_loss': 0.746590533753041, 'epoch': 3.0})

In [ ]:
trainer.save_model("test-squad-trained1")

Saving model checkpoint to test-squad-trained1
Configuration saved in test-squad-trained1/config.json
Model weights saved in test-squad-trained1/pytorch_model.bin
tokenizer config file saved in test-squad-trained1/tokenizer_config.json
Special tokens file saved in test-squad-trained1/special_tokens_map.json


# **evaluate**

In [ ]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [ ]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 201]), torch.Size([16, 201]))

In [ ]:
n_best_size = 20

In [ ]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

In [ ]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
validation_features = val_datasets.map(
    prepare_validation_features,
    batched=True,
    remove_columns=val_datasets.column_names
)

  0%|          | 0/15 [00:00<?, ?ba/s]

In [ ]:
raw_predictions = trainer.predict(validation_features)

The following columns in the test set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id.
***** Running Prediction *****
  Num examples = 24883
  Batch size = 16


In [ ]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [ ]:
max_answer_length = 30

In [ ]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = val_datasets[0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 17.165253, 'text': 'والیبال'},
 {'score': 9.694532, 'text': 'والی'},
 {'score': 7.293938, 'text': 'بال'},
 {'score': 7.135642,
  'text': 'والیبال یک ورزش گروهی و ششمین ورزش پر طرفدار و گسترده\u200cترین ورزش'},
 {'score': 6.7142277,
  'text': 'والیبال یک ورزش گروهی و ششمین ورزش پر طرفدار و گسترده\u200cترین ورزش در بعضی کشورهای جهان است'},
 {'score': 5.8589587, 'text': 'والیبال یک ورزش گروهی و ششمین'},
 {'score': 5.152854,
  'text': 'والیبال یک ورزش گروهی و ششمین ورزش پر طرفدار و گسترده\u200cترین ورزش در بعضی کشورهای جهان'},
 {'score': 4.835779, 'text': 'والیبال یک ورزش گروهی'},
 {'score': 4.5467925,
  'text': 'والیبال یک ورزش گروهی و ششمین ورزش پر طرفدار و گسترده\u200cترین ورزش در بعضی کشورهای جهان است که در آن'},
 {'score': 4.374149, 'text': 'والیبال یک'},
 {'score': 4.3336654, 'text': 'والیبال یک ورزش گروهی و ششمین ورزش پر طرفدار'},
 {'score': 3.7883325, 'text': 'والیبال یک ورزش گروهی و شش'},
 {'score': 3.0414352, 'text': 'والیبال یک ورزش'},
 {'score': 2.8507867, 'text': '

In [ ]:
import collections

examples = val_datasets
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [ ]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()
    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")
    # Let's loop over all the examples!
    print(len(examples))
    new_example = []
    ids = []
    for example_index, example in enumerate(tqdm(examples)):
        
        i += 1
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        # if not squad_v2:
        # print(example["id"],example["question"],example["answers"])
        try:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer
        except TypeError:
          continue
        # else:
        #     answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        #     predictions[example["id"]] = answer
    print(i)
    return predictions,new_example

In [ ]:
validation_features

Dataset({
    features: ['attention_mask', 'example_id', 'input_ids', 'offset_mapping', 'token_type_ids'],
    num_rows: 24883
})

In [ ]:
val_datasets

Dataset({
    features: ['answers', 'context', 'id', 'question', 'title'],
    num_rows: 14312
})

In [ ]:
final_predictions,new_example = postprocess_qa_predictions(val_datasets, validation_features, raw_predictions.predictions)

Post-processing 14312 example predictions split into 24883 features.
14312


  0%|          | 0/14312 [00:00<?, ?it/s]

39194


In [ ]:
from datasets import load_metric

In [ ]:
metric = load_metric("squad_v2" )

Downloading:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

In [ ]:
len(final_predictions.items())

14312

In [ ]:
len(val_datasets)

14312

In [ ]:
formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in new_example]
metric.compute(predictions=formatted_predictions, references=references)

{'HasAns_exact': 68.66260308354249,
 'HasAns_f1': 77.60473540328479,
 'HasAns_total': 8367,
 'NoAns_exact': 0.0,
 'NoAns_f1': 0.0,
 'NoAns_total': 5945,
 'best_exact': 51.64896590273896,
 'best_exact_thresh': 0.0,
 'best_f1': 53.2431292700296,
 'best_f1_thresh': 0.0,
 'exact': 40.14114030184461,
 'f1': 45.3688388149304,
 'total': 14312}

In [ ]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("sgugger/my-awesome-model")

HTTPError: ignored